In [1]:
import OLS_regression_analysis_new
import bbgclient
import datetime
import dfutils
import pandas as pd

In [7]:
metrics = ['P/E', "EV/EBITDA", "EV/Sales", 'DVD Yield', 'FCF Yield']
alpha_ticker = 'JACK US EQUITY'
analyst_upside = 98
analyst_downside = 73
analyst_pt_wic = 88
lookback = 120
unaffected_date = '2017-05-16'
target_date = "2019-02-21"
api_host = bbgclient.bbgclient.get_next_available_host()
fperiod = "1BF"

In [8]:
OLS_regression_analysis_new.bloomberg_peers("JACK US EQUITY")

['MCD US EQUITY',
 'WEN US EQUITY',
 'YUM US EQUITY',
 'LOCO US EQUITY',
 'YUMC US EQUITY',
 'QSR CN EQUITY']

In [9]:
results = OLS_regression_analysis_new.run_regression_optimal_peers(alpha_ticker, unaffected_date, lookback_period = 120)

In [10]:
results

({'P/E': {'MCD US EQUITY': -0.6547756043537755,
   'WEN US EQUITY': 0.37462814956534457,
   'YUM US EQUITY': -0.34444053365822525,
   'LOCO US EQUITY': 0.33070785834113925,
   'YUMC US EQUITY': -0.08722756634491818,
   '(Intercept)': 27.2978193177026},
  'EV/EBITDA': {'MCD US EQUITY': -0.27923586403912704,
   'WEN US EQUITY': 0.1944470719612887,
   'LOCO US EQUITY': 0.5736259842543897,
   'YUMC US EQUITY': -0.17367176979797408,
   '(Intercept)': 8.970655769478123},
  'EV/Sales': {'WEN US EQUITY': 0.3342208652028911,
   'YUM US EQUITY': -0.21085478598188387,
   'QSR CN EQUITY': 0.19913429453699316,
   '(Intercept)': 0.7771808754094983},
  'DVD Yield': {'MCD US EQUITY': -0.26342622295893103,
   'WEN US EQUITY': 0.5367546868081103,
   'YUM US EQUITY': -0.02244112149128218,
   '(Intercept)': 1.3078452678455268},
  'FCF Yield': {'MCD US EQUITY': -0.3006376695581049,
   'WEN US EQUITY': 0.7022377820172085,
   'LOCO US EQUITY': 0.2074864539848666,
   'QSR CN EQUITY': -0.21844955409410574,
   

In [11]:
def run_OLS_regression_analysis(alpha_ticker, tgtDate, analyst_upside, analyst_downside, analyst_pt_wic, regression_results, api_host, adjustments_df_now = None, adjustments_df_ptd = None, f_period = '1BF'):
    metrics = ['P/E', "EV/EBITDA", "EV/Sales", 'DVD Yield', 'FCF Yield']
    slicer = dfutils.df_slicer()
    price_tgt_dt = datetime.datetime.strptime(tgtDate, '%Y-%m-%d')
    as_of_dt = datetime.datetime.today()
    
    if len(regression_results) == 2:
        optimal_results = regression_results[0]
        overlap_results = regression_results[1]
    else:
        optimal_results = regression_results
        
    peers = []
    for metric in metrics:  
        results = optimal_results[metric]
        for x in optimal_results[metric]:
            if x not in peers:
                peers.append(x)
    peers.remove('(Intercept)')
    
    peer2ptd_multiple = {p:OLS_regression_analysis_new.multiples_df(p,slicer.prev_n_business_days(100,price_tgt_dt).strftime('%Y%m%d'),price_tgt_dt.strftime('%Y%m%d'),api_host,fperiod='1BF',multiples_to_query=metrics) for p in peers}
    peer2now_multiple = {p:OLS_regression_analysis_new.multiples_df(p,slicer.prev_n_business_days(100,as_of_dt).strftime('%Y%m%d'),as_of_dt.strftime('%Y%m%d'),api_host,fperiod='1BF',multiples_to_query=metrics) for p in peers}
    
    dataframes = {
        'peer2ptd_multiple': peer2ptd_multiple,
        'peer2now_multiple': peer2now_multiple
    }
    print(dataframes['peer2ptd_multiple'])
    
    #original_results = OLS_regression_analysis_new.premium_analysis_df_OLS_quick(dataframes, regression_results, alpha_ticker, analyst_upside, tgtDate, api_host, fperiod = "1BF")

    adjusted_results = OLS_regression_analysis_new.premium_analysis_df_OLS2_quick(dataframes, regression_results, alpha_ticker, analyst_upside, analyst_downside, analyst_pt_wic, tgtDate, api_host, adjustments_df_now = None, adjustments_df_ptd = None, fperiod = "1BF")
    
    return {'adjusted_results': adjusted_results
           }

In [12]:
run_OLS_regression_analysis(alpha_ticker, target_date, analyst_upside, analyst_downside, analyst_pt_wic, results, api_host, adjustments_df_now = None, adjustments_df_ptd = None, f_period = '1BF')

{'MCD US EQUITY':           Date      PX  FCF Yield     P/E  EV/EBITDA  EV/Sales  DVD Yield
0   2018-10-02  165.18   0.040040  20.495     14.718     7.545     2.8091
1   2018-10-03  164.66   0.040288  20.428     14.676     7.526     2.8179
2   2018-10-04  165.80   0.040015  20.567     14.757     7.568     2.7986
3   2018-10-05  166.57   0.039846  20.655     14.808     7.595     2.7856
4   2018-10-08  167.77   0.039571  20.797     14.893     7.640     2.7657
5   2018-10-09  169.83   0.039268  21.039     15.014     7.715     2.7321
6   2018-10-10  168.37   0.039618  20.852     14.906     7.661     2.7558
7   2018-10-11  162.97   0.040927  20.185     14.517     7.462     2.8471
8   2018-10-12  163.82   0.040712  20.286     14.577     7.494     2.8324
9   2018-10-15  163.67   0.040782  20.270     14.560     7.494     2.8350
10  2018-10-16  164.07   0.040619  20.323     14.595     7.518     2.8281
11  2018-10-17  166.77   0.039970  20.652     14.788     7.619     2.7823
12  2018-10-18  166.

{'adjusted_results': {'optimal_peer_results': {'P/E': {'Alpha Downside': 65.10287239951259,
    'Alpha PT WIC': 80.10287239951259,
    'Alpha Upside': 90.10287239951259},
   'EV/EBITDA': {'Alpha Downside': 76.62332367388268,
    'Alpha PT WIC': 91.62332367388268,
    'Alpha Upside': 101.62332367388268},
   'EV/Sales': {'Alpha Downside': 80.21135725108459,
    'Alpha PT WIC': 95.21135725108459,
    'Alpha Upside': 105.21135725108459},
   'DVD Yield': {'Alpha Downside': 73.04100011120237,
    'Alpha PT WIC': 88.04100011120237,
    'Alpha Upside': 98.04100011120237},
   'FCF Yield': {'Alpha Downside': 67.8666185654601,
    'Alpha PT WIC': 82.8666185654601,
    'Alpha Upside': 92.8666185654601}},
  'overlap_peer_results': {'P/E': {'Alpha Downside': 76.38645598355295,
    'Alpha PT WIC': 91.38645598355295,
    'Alpha Upside': 101.38645598355295},
   'EV/EBITDA': {'Alpha Downside': 85.25138109274803,
    'Alpha PT WIC': 100.25138109274803,
    'Alpha Upside': 110.25138109274803},
   'EV/Sale